In [ ]:

import os
import pandas as pd

def createdataframe(directory):
    filenames = []
    labels = []

    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)
        if os.path.isdir(label_path):
            for filename in os.listdir(label_path):
                if filename.endswith(".jpg"):
                    filenames.append(os.path.join(label_path, filename))
                    labels.append(label)

    dataframe = pd.DataFrame({
        'image': filenames,
        'label': labels
    })

    return dataframe

TRAIN_DIR = 'C:\\Users\\rohit\\Downloads\\archive\\images\\train'
train = createdataframe(TRAIN_DIR)

In [ ]:
print(train)

In [12]:

import os
import pandas as pd

def createdataframe(directory):
    filenames = []
    labels = []

    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)
        if os.path.isdir(label_path):
            for filename in os.listdir(label_path):
                if filename.endswith(".jpg"):
                    filenames.append(os.path.join(label_path, filename))
                    labels.append(label)

    dataframe = pd.DataFrame({
        'image': filenames,
        'label': labels
    })

    return dataframe

TEST_DIR = 'C:\\Users\\rohit\\Downloads\\archive\\images\\test'
test = createdataframe(TEST_DIR)

In [ ]:
print(test)
print(test['image'])

from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
from keras.preprocessing.image import load_img, img_to_array

def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale', target_size=(48, 48))
        img_array = img_to_array(img)
        features.append(img_array)
    return features


In [ ]:
train_features = extract_features(train['image'])

In [ ]:
test_features = extract_features(test['image'])

In [16]:
import numpy as np

train_features = np.array(train_features)
test_features = np.array(test_features)

x_train = train_features / 255.0
x_test = test_features / 255.0


In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [20]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 128

train_generator = train_datagen.flow_from_directory('C:\\Users\\rohit\\Downloads\\archive\\images\\train', target_size=(48, 48), batch_size=128, class_mode='categorical', color_mode='grayscale')
val_generator = val_datagen.flow_from_directory('C:\\Users\\rohit\\Downloads\\archive\\images\\test', target_size=(48, 48), batch_size=128, class_mode='categorical', color_mode='grayscale')

# Build the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(48, 48, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit(train_generator, epochs=100, validation_data=val_generator)

model.save('face_emotions.h5')


In [22]:
from tensorflow.keras.models import load_model

emotion_model = load_model('face_emotions.h5')
